<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Objetive" data-toc-modified-id="Objetive-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Objetive</a></span></li><li><span><a href="#Experiment-Overview" data-toc-modified-id="Experiment-Overview-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Experiment Overview</a></span></li></ul></div>

# Explanations 

## Intro


Modern day machine learning models can be very complex, having even millions of parameters and many mathematical operations coupling the input variables. For instance a model such as Rnadom Forest can contain hundreds of decision trees with variate deep that are combined to produce a single output number. Similarly, neural networks might have many layers that couple the input variable using a collecion of mathematical operations. In simpler models such as logistic regression, one can easily infer the impact of a particular feature looking at the related coeficient, such approach is impossible for hifh performance models such as multi-layer neural networks and random forest. 

In  high stakes situations such as credit risk modelling it is essencial not only produce good models but also to be able verify how the input features impact the predictions. For complex models, one can tackle this challege using the so called *explainers*. Many explainers are simpler models that locally approximate the predictions of more complex models. The SHAP (Shapley Additive Explanations) explainer lib, for instance, use ideias from game theory to quantify the impact of each feature. 

In mathematical terms, we want to undertand the impact of the vector input $x$ for the prediction produced by a complex model $f$ using a explanation model $g$.  This explanation model will be a linear function of a simplified binary input $z$, that is

$$
g(z) = \phi_0 + \sum_{i=1}^N \phi_i z_i   
$$
where $z\in \{0,1\}^N$, N is the number of features, and $\phi_i \in \mathcal{R}$ is the feature impact. 




# Imports

In [35]:
# basic imports

import numpy as np
import pandas as pd


# specifc imports

import optuna 


import plotly.express as px


# Load datasets

In [2]:
########################
# DATA FILE NAME IN CSV
datafile='../data/polish_companies_bankruptcy_1o_year.csv'
########################

df = pd.read_csv(datafile)

print(df.shape)

print(df.shape)

features = list(df.columns)
features.remove('class')

df.head()

(7027, 65)
(7027, 65)


,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X56,X57,X58,X59,X60,X61,X62,X63,X64,class
0,0.200550,0.37951,0.39641,2.0472,32.3510,0.38825,0.249760,1.33050,1.1389,0.50494,...,0.121960,0.39718,0.87804,0.001924,8.4160,5.1372,82.658,4.4158,7.4277,0
1,0.209120,0.49988,0.47225,1.9447,14.7860,0.00000,0.258340,0.99601,1.6996,0.49788,...,0.121300,0.42002,0.85300,0.000000,4.1486,3.2732,107.350,3.4000,60.9870,0
2,0.248660,0.69592,0.26713,1.5548,-1.1523,0.00000,0.309060,0.43695,1.3090,0.30408,...,0.241140,0.81774,0.76599,0.694840,4.9909,3.9510,134.270,2.7185,5.2078,0
3,0.081483,0.30734,0.45879,2.4928,51.9520,0.14988,0.092704,1.86610,1.0571,0.57353,...,0.054015,0.14207,0.94598,0.000000,4.5746,3.6147,86.435,4.2228,5.5497,0
4,0.187320,0.61323,0.22960,1.4063,-7.3128,0.18732,0.187320,0.63070,1.1559,0.38677,...,0.134850,0.48431,0.86515,0.124440,6.3985,4.3158,127.210,2.8692,7.8980,0


In [3]:
# feature description

fdescr  = open('../data/polish_data_description.txt','r')

feature_description = {}
for line in fdescr.readlines():
    d = line.split(' ')
    feature_description[d[0]] = ' '.join(d[1:]).replace('\n','')

In [4]:
for k,v in feature_description.items():
    print(k,': ',v)

X1 :  net profit / total assets
X2 :  total liabilities / total assets
X3 :  working capital / total assets
X4 :  current assets / short-term liabilities
X5 :  [(cash + short-term securities + receivables - short-term liabilities) / (operating expenses - depreciation)] * 365
X6 :  retained earnings / total assets
X7 :  EBIT / total assets
X8 :  book value of equity / total liabilities
X9 :  sales / total assets
X10 :  equity / total assets
X11 :  (gross profit + extraordinary items + financial expenses) / total assets
X12 :  gross profit / short-term liabilities
X13 :  (gross profit + depreciation) / sales
X14 :  (gross profit + interest) / total assets
X15 :  (total liabilities * 365) / (gross profit + depreciation)
X16 :  (gross profit + depreciation) / total liabilities
X17 :  total assets / total liabilities
X18 :  gross profit / total assets
X19 :  gross profit / sales
X20 :  (inventory * 365) / sales
X21 :  sales (n) / sales (n-1)
X22 :  profit on operating activities / total ass

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X = df[features]
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [7]:
print('train p(b) = %.3f ' %y_train.mean() )
print('test p(b) = %.3f ' %y_test.mean() )

train p(b) = 0.041 
test p(b) = 0.034 


Definetily a imbalanced case.

# Experiment



In [8]:
import optuna

# xgboost

In [9]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold

In [49]:
def objective_xgb(trial, X, y):
    """Define objective function for hyperparameter optimization
    Parameters
    ----------
    trial:
    data:
    model:
    
    Returns
    -------
    metric: float
    """

    # Suggest values of the hyperparameters using a trial object.
    params = {'eval_metric': 'logloss',
                'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 1.0),
                  'max_depth': trial.suggest_int('max_depth', 1, 20),
                  'n_estimators': trial.suggest_int('n_estimators', 1, 1000),
#                  'criterion' : trial.suggest_categorical('criterion' ,['gini', 'entropy'])
                  'reg_alpha': 0.5,
                  'reg_lambda': 0.5,
                  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 1e-4, 1),
#                   'early_stopping_rounds': 20,
                  'min_child_weight': trial.suggest_int('min_child_weight', 2,10),
                  'gamma': trial.suggest_int('gamma', 3, 8)}
    
    base_model = xgb.XGBClassifier(n_jobs=10, 
                                   tree_method='hist', 
                                   predictor='cpu_predictor',
                                   use_label_encoder=False)

    model = base_model.set_params(**params)
    
    skf = StratifiedKFold(n_splits=10, shuffle = True)
    
    m1_scores = cross_val_score(model, X, y, scoring= 'roc_auc', cv=skf.split(X, y,), n_jobs=10)
    m2_scores = cross_val_score(model, X, y, scoring= 'average_precision', cv=skf.split(X, y), n_jobs=10)  
    
    # see https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    
    return np.median(m1_scores), np.median(m2_scores)

In [50]:

study_2o = optuna.create_study(study_name= "auc_prec", 
                                    directions= ['maximize','maximize'],
                                    pruner = optuna.pruners.MedianPruner(
                                    n_startup_trials=10, n_warmup_steps=30, interval_steps=10),
                                    sampler = optuna.samplers.NSGAIISampler(),)


[I 2022-02-25 10:38:12,655] A new study created in memory with name: auc_prec


In [61]:
study_2o.optimize(lambda trial: objective_xgb(trial, 
                                       X = X_train,
                                       y = y_train),
                                       n_trials=20)

[I 2022-02-25 13:53:21,536] Trial 60 finished with values: [0.901906779661017, 0.6358081521266328] and parameters: {'learning_rate': 0.0027657170796970454, 'max_depth': 2, 'n_estimators': 476, 'colsample_bytree': 0.6860694372449941, 'min_child_weight': 3, 'gamma': 5}. 
[I 2022-02-25 13:53:22,829] Trial 61 finished with values: [0.8496284501061572, 0.34785486930497556] and parameters: {'learning_rate': 0.002192949949809065, 'max_depth': 15, 'n_estimators': 14, 'colsample_bytree': 0.32129390677560454, 'min_child_weight': 9, 'gamma': 6}. 
[I 2022-02-25 13:53:47,335] Trial 62 finished with values: [0.9388262891791717, 0.7325281966452564] and parameters: {'learning_rate': 0.19641360933473742, 'max_depth': 12, 'n_estimators': 492, 'colsample_bytree': 0.7369213813129891, 'min_child_weight': 5, 'gamma': 4}. 
[I 2022-02-25 13:54:11,484] Trial 63 finished with values: [0.9305371504912014, 0.6939929277685875] and parameters: {'learning_rate': 0.0357091631443731, 'max_depth': 12, 'n_estimators': 4

In [59]:
# study = pickle.load( open( "model_study.pickle", "rb" ) )

df_study = study_2o.trials_dataframe()

df_study.sample(3)

,number,values_0,values_1,datetime_start,datetime_complete,duration,params_colsample_bytree,params_gamma,params_learning_rate,params_max_depth,params_min_child_weight,params_n_estimators,system_attrs_nsga2:generation,system_attrs_nsga2:parents,state
18,18,0.931462,0.718120,2022-02-25 10:46:28.283668,2022-02-25 10:47:05.415086,0 days 00:00:37.131418,0.964507,6,0.044551,12,10,840,0,NaN,COMPLETE
48,48,0.869677,0.561742,2022-02-25 12:52:08.050396,2022-02-25 12:52:19.669441,0 days 00:00:11.619045,0.180574,8,0.002449,5,10,421,0,NaN,COMPLETE
7,7,0.945286,0.750968,2022-02-25 10:40:58.601770,2022-02-25 10:41:36.787925,0 days 00:00:38.186155,0.405962,3,0.050204,7,4,795,0,NaN,COMPLETE


In [60]:

fig = px.scatter(df_study,
                 x='values_0',y='values_1',
#                  error_x='values_1',
#                  color='params_classifier_method',
                 size='params_max_depth',
#                  symbol="sampler",
                 
                 labels={'params_n_estimators':'n_estimators',
                         'params_max_depth':'max_depth',
                         'params_classifier_method':'classifier',
                         'params_imputer_method':'imputer',
                          'values_0':'roc_auc',
                          'values_1':'avg_prec'},
                 hover_name='number',
                 color_continuous_scale='bluered')

# fig.add_vline(x=basemodel_logloss,
#              annotation_text="model baseline", 
#              annotation_position="top right")

fig.update_layout(
    title="Multi-Objective Optimization",
#     xaxis_title="FPR",
#     yaxis_title="TPR",
    font_size=18)
    
fig.show()

# Conclusions

Summaize here the main results